<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/48_Acumulador_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Acumulador PySpark
- Exemplo 01
- Date: 2025-03-25

In [6]:
import datetime
dt = datetime.datetime.now()
print(dt)


2025-03-25 14:09:55.188462


In [28]:
from pyspark.sql import SparkSession
# Iniciando Sessão
spark = SparkSession.builder.appName("Accumulator Example").getOrCreate()
# Criando acumulador
invalid_count = spark.sparkContext.accumulator(0)
# Example dados
data = [
    "Luciana,25,M",
    "Angelica,30,F",
    "Helena,3,F",
    "Bob,40,M",
    "Alice,28,F",
    "Charlie,,M",

]
# Parallelize the data
rdd = spark.sparkContext.parallelize(data)

# Cada registro de dados será processado, e se ele não cumprir os critérios, o invalid_count é incrementado.
def process_record(record):
    global invalid_count
    fields = record.split(",")
    if len(fields) != 3 or not fields[1].isdigit():
        invalid_count += 1
        return None
    return record

# Processo RDD


processed_rdd = rdd.map(process_record).filter(lambda x: x is not None)


processed_rdd.collect()


print(f"Number of invalid records: {invalid_count.value}")
# Stop the Spark session
spark.stop()

Number of invalid records: 1


- Acumulador PySpark
- Exemplo 02

### Como contar as Linhas usando acumulador
- O collect a ação desencadeia a transformação, fazendo com que o acumulador agregue valores das tarefas.

In [18]:
from pyspark import SparkContext

sc = SparkContext("local", "Acumulador Exemplo")

# Cria um acumulador numérico inicializado com 0
acumulador = sc.accumulator(0)

# Função para contar as linhas
def contar_linhas(linha):
    global acumulador
    acumulador += 1
    return linha

dados = sc.parallelize(["linha 1", "linha 2", "linha 3"])

# Usando foreach
dados.foreach(contar_linhas)

# Obtém o valor final do acumulador
print("Número de linhas:", acumulador.value)

sc.stop()


Número de linhas: 3


In [19]:
from pyspark import SparkContext

sc = SparkContext("local", "Acumulador Exemplo")

# Cria um acumulador numérico inicializado com 0
acumulador = sc.accumulator(0)

def contar_linhas(linha):
    global acumulador
    acumulador += 1
    return linha

dados = sc.parallelize(["linha 1", "linha 2", "linha 3"])
dados.foreach(contar_linhas)

# Obtém o valor final do acumulador
print("Número de linhas:", acumulador.value)

sc.stop()

Número de linhas: 3


Exemplo 03 de Acumuladores no Spark
- verificar "erro" em uma linha


In [20]:
from pyspark.sql import SparkSession

# Inicia a sessão Spark
spark = SparkSession.builder.appName("Exemplo03").getOrCreate()

# Cria um acumulador
erro_count = spark.sparkContext.accumulator(0)

# Função que conta linhas com erro (exemplo simples)
def verificar_linha(linha):
    if "ERRO" in linha:
        erro_count.add(1)
    return linha

# Exemplo de RDD
dados = ["Linha ok", "ERRO: Falha no sistema", "Linha normal", "ERRO: Timeout"]
rdd = spark.sparkContext.parallelize(dados)

# Aplica a função no RDD
rdd.map(verificar_linha).collect()

# Mostra o resultado do acumulador
print(f"Total de linhas com erro: {erro_count.value}")


Total de linhas com erro: 2


- Exemplo 04


In [27]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.accumulators import AccumulatorParam

# Inicia Spark
spark = SparkSession.builder.appName("Exemplo04").getOrCreate()
sc = spark.sparkContext

# Define um acumulador de lista personalizado
# Implement custom accumulator logic using AccumulatorParam instead of AccumulatorV2
class ListAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return []

    def addInPlace(self, value1, value2):
        value1.extend(value2)  # Extend the list with new values
        return value1

# Registra o acumulador using AccumulatorParam
lista_acumulada = sc.accumulator([], ListAccumulatorParam())

# Função para adicionar palavras ao acumulador
def adicionar_palavra(palavra):
    global lista_acumulada
    lista_acumulada += [palavra]

# Cria o RDD com palavras
dados = sc.parallelize(["palavra1", "palavra2", "palavra3"])

# Aplica a função para adicionar cada palavra ao acumulador
dados.foreach(adicionar_palavra)

# Exibe o resultado final
print("Lista de palavras:", lista_acumulada.value)

sc.stop()

Lista de palavras: ['palavra1', 'palavra2', 'palavra3']
